In [1]:
import os
import time
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement

In [2]:
# URL = 'https://www.skyscanner.ru'
URL = 'https://www.skyscanner.ru/transport/flights/mosc/ber/211120'
dump_path = Path(os.environ['PWD']) / 'app/notebook/dump.html'

In [3]:
def dump_HTML(driver: WebDriver) -> None:
    with open(dump_path, 'w') as file:
        file.write(driver.page_source)

In [4]:
def check_and_handle_captcha(driver: WebDriver) -> None:
    time.sleep(2)
    
    if driver.find_elements(By.ID, 'px-captcha'):
        print('--- CAPTCHA DETECTED ---')
        while driver.find_elements(By.CLASS_NAME, 'px-loader-wrapper'):
            time.sleep(2)
        
        while driver.find_elements(By.ID, 'px-captcha'):
            captcha: WebElement = driver.find_element(By.ID, 'px-captcha')
            time.sleep(2)
            ActionChains(driver) \
                .move_to_element(captcha) \
                .click_and_hold(captcha) \
                .pause(5) \
                .release() \
                .perform()

        driver.save_screenshot('captcha_handler_result.png')
        time.sleep(2)

In [5]:
options = Options()
options.headless = True
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-web-security')
options.add_argument('--no-sandbox')
options.add_argument('--user-agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"')
options.add_argument('--window-size=1920,1080')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

In [6]:
driver: WebDriver = webdriver.Chrome(
    options=options,
    service=Service('/usr/bin/chromedriver')
)

try:
    driver.get(URL)
    check_and_handle_captcha(driver)
    driver.save_screenshot('main.png')
    
    # ждём, пока пропадёт индикатор продолжающейся загрузки
    while driver.find_elements(By.CLASS_NAME, 'SummaryInfo_progressTextContainer__Njg5Y'):
        time.sleep(10)
        
    check_and_handle_captcha(driver)
    driver.save_screenshot('main2.png')
    
    # переходим на дно страницы
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    check_and_handle_captcha(driver)
    # кликаем на кнопку "Показать больше"
    driver.find_element(
        By.XPATH,
        '/html/body/div[3]/div[3]/div/div[2]/div[1]/div/div/div/div[1]/button'
    ).click()
    check_and_handle_captcha(driver)
    driver.save_screenshot('main3.png')
    
    # элемент с карточками рейсов
    results_div: WebElement = driver.find_element(
        By.XPATH,
        '/html/body/div[3]/div[3]/div/div[2]/div[1]/div/div/div/div[1]/div[3]'
    )
finally:
    driver.quit()

--- CAPTCHA DETECTED ---
